In [ ]:
import math

import sympy  # For primality testing

def find_potential_prime(n):
    # Create a list of all integers from 2 to sqrt(n)
    upper_bound = n
    numbers = list(range(2, upper_bound))
    
    # Create a set of remainders when n is divided by each number in the list
    remainders = set()
    for i in numbers:
        remainders.add(i - (n % i))
    
    # Find the first missing even number in the set if the current number is odd, or odd if even
    m = 1 if n % 2 == 0 else 2
    while m in remainders:
        m += 2
    
    # Create the potential prime number
    primetest = n + m
    
    # Check if the potential prime is actually prime
    is_prime = sympy.isprime(primetest)
    
    return primetest, is_prime

# Example usage
n = 2222
primetest, is_prime = find_potential_prime(n)
print(f"For n = {n}:")
print(f"First missing even remainder: {primetest - n}")
print(f"Potential prime number: {primetest}")
print(f"Is it prime? {is_prime}")

In [ ]:
import math

import sympy  # For primality testing


def find_potential_prime(n):
    # Create a list of all integers from 2 to sqrt(n)
    upper_bound = math.ceil(math.sqrt(n))
    numbers = list(range(2, upper_bound))
    
    # Create a set of remainders when n is divided by each number in the list
    remainders = set()
    for i in numbers:
        remainders.add(i - (n % i))
    
    # Find the first missing even number in the set if the current number is odd, or odd if even
    m = 1 if n % 2 == 0 else 2
    while m in remainders:
        m += 2
    
    # Create the potential prime number
    primetest = n + m
    
    # Check if the potential prime is actually prime
    is_prime = sympy.isprime(primetest)
    
    return primetest, is_prime

# Test find_potential_prime from 1 to max, collect wrong guesses
max_n = 50000  # You can adjust this value as needed
wrong_guesses = set()
progress_step = max_n // 100 if max_n >= 100 else 1
for n in range(1, max_n + 1):
    primetest, is_prime = find_potential_prime(n)    
    if not is_prime:
        wrong_guesses.add(primetest)
    if n % progress_step == 0 or n == max_n:
        print(f"Checked up to n={n} / {max_n}", end='\r', flush=True)
print()
print(f"Potential primes that were not actually prime (n=1 to {max_n}):")
print(sorted(wrong_guesses))

In [79]:
import math
import sympy
from collections import defaultdict

def find_potential_prime(n):
    upper_bound = math.ceil(math.sqrt(n))
    numbers = list(range(2, upper_bound))
    remainders = set()
    for i in numbers:
        remainders.add(i - (n % i))
    m = 1 if n % 2 == 0 else 2
    while m in remainders:
        m += 2
    primetest = n + m
    is_prime = sympy.isprime(primetest)
    return primetest, is_prime, m, numbers, remainders

def is_square(n):
    root = int(n**0.5)
    return root * root == n

def is_cube(n):
    root = int(round(n**(1/3)))
    return root ** 3 == n

# Compute wrong guesses independently and track n, m, i values, correct m, missing i, correct next prime, extended i, missing in remainders, and i values for correct m
max_n = 20000  # You can adjust this value as needed
wrong_guesses = set()
wrong_guess_n_map = defaultdict(list)  # Maps wrong guess to list of n values
wrong_guess_nm_map = defaultdict(list)  # Maps wrong guess to list of (n, m) tuples
wrong_guess_nmi_map = defaultdict(list)  # Maps wrong guess to list of (n, m, i_values) tuples
wrong_guess_full_map = defaultdict(list)  # Maps wrong guess to list of (n, m, i_values, correct_m, missing_i, correct_next_prime, extended_i, correct_m_missing_in_remainders, correct_m_i_values) tuples
progress_step = max_n // 100 if max_n >= 100 else 1
for n in range(1, max_n + 1):
    primetest, is_prime, m, numbers, remainders = find_potential_prime(n)
    if not is_prime:
        wrong_guesses.add(primetest)
        wrong_guess_n_map[primetest].append(n)
        wrong_guess_nm_map[primetest].append((n, m))
        # Find i values that caused m to be skipped (i where i - (n % i) == m)
        i_values = [i for i in numbers if i - (n % i) == m]
        wrong_guess_nmi_map[primetest].append((n, m, i_values))
        # Find the correct m (the lowest m not in remainders and that would yield a prime)
        correct_m = m
        missing_i = None
        correct_next_prime = None
        extended_i = None
        correct_m_missing_in_remainders = False
        correct_m_i_values = []
        test_m = 1
        while True:
            candidate = n + test_m
            if sympy.isprime(candidate):
                correct_m = test_m
                correct_next_prime = candidate
                # Find the i that would have given this m: i = (test_m + n % i) for i in numbers, but i must be in range(2, n)
                # Actually, for a given m, i = m + (n % i), but we want i such that i - (n % i) == correct_m
                # So, for all possible i in range(2, n), check if i - (n % i) == correct_m
                possible_i = [i for i in range(2, n) if i - (n % i) == correct_m]
                missing_i = possible_i[0] if possible_i else None
                # If missing_i is still None, search up to max_n for i
                if missing_i is None:
                    for i in range(n, max_n + 1):
                        if i - (n % i) == correct_m:
                            extended_i = i
                            break
                # Check if correct_m is another missing value in the set of remainders
                if correct_m not in remainders:
                    correct_m_missing_in_remainders = True
                    # Get i values associated with correct_m
                    correct_m_i_values = [i for i in range(2, max_n + 1) if i - (n % i) == correct_m]
                break
            test_m += 1

        wrong_guess_full_map[primetest].append((n, m, i_values, correct_m, missing_i, correct_next_prime, extended_i, correct_m_missing_in_remainders, correct_m_i_values))
    if n % progress_step == 0 or n == max_n:
        print(f"Checked up to n={n} / {max_n}", end='\r', flush=True)
print()

def is_palindrome(n):
    return str(n) == str(n)[::-1]

def is_highly_composite(n):
    num_divisors = len(sympy.divisors(n))
    for k in range(1, n):
        if len(sympy.divisors(k)) >= num_divisors:
            return False
    return True

def is_semiprime(n):
    factors = sympy.factorint(n)
    return sum(factors.values()) == 2

def is_powerful(n):
    factors = sympy.factorint(n)
    return all(exp >= 2 for exp in factors.values()) if factors else False

number_types = defaultdict(list)
for num in sorted(wrong_guesses):
    types = []
    if is_square(num):
        types.append('Perfect square')
    if is_cube(num):
        types.append('Perfect cube')
    if is_semiprime(num):
        types.append('Semiprime')
    if is_powerful(num):
        types.append('Powerful')
    if is_highly_composite(num):
        types.append('Highly composite')
    if is_palindrome(num):
        types.append('Palindromic')
    if num % 2 == 0:
        types.append('Even')
    else:
        types.append('Odd')
    for t in types:
        number_types[t].append(num)

print("Number type counts among wrong guesses:")
for t, nums in number_types.items():
    print(f"{t}: {len(nums)}")
print("\nExample numbers for each type:")
for t, nums in number_types.items():
    print(f"{t}: {nums[:10]}")

# Print mapping of wrong guesses to n values
print("\nWrong guess mapping (first 10 wrong guesses):")
for wrong, n_list in list(wrong_guess_n_map.items())[:10]:
    print(f"Wrong guess {wrong}: n values {n_list[:10]}")

# Print mapping of wrong guesses to (n, m) tuples
print("\nWrong guess n, m mapping (first 10 wrong guesses):")
for wrong, nm_list in list(wrong_guess_nm_map.items())[:10]:
    print(f"Wrong guess {wrong}: (n, m) tuples {nm_list[:10]}")

# Print mapping of wrong guesses to (n, m, i_values) tuples
print("\nWrong guess n, m, i mapping (first 10 wrong guesses):")
for wrong, nmi_list in list(wrong_guess_nmi_map.items())[:10]:
    print(f"Wrong guess {wrong}: (n, m, i_values) tuples {nmi_list[:3]}")

# Print mapping of wrong guesses to (n, m, i_values, correct_m, missing_i, correct_next_prime, extended_i, correct_m_missing_in_remainders, correct_m_i_values) tuples
print("\nWrong guess full mapping (first 10 wrong guesses):")
for wrong, full_list in list(wrong_guess_full_map.items())[:10]:
    for tup in full_list[:3]:
        n, m, i_values, correct_m, missing_i, correct_next_prime, extended_i, correct_m_missing_in_remainders, correct_m_i_values = tup
        print(f"Wrong guess {wrong}: n={n}, m={m}, i_values={i_values}, correct_m={correct_m}, missing_i={missing_i}, correct_next_prime={correct_next_prime}, extended_i={extended_i}, correct_m_missing_in_remainders={correct_m_missing_in_remainders}, correct_m_i_values={correct_m_i_values}")
# Final print to ensure output is flushed
print()

Checked up to n=20000 / 20000hecked up to n=4600 / 20000Checked up to n=4800 / 20000Checked up to n=6800 / 20000Checked up to n=7000 / 20000Checked up to n=10400 / 20000Checked up to n=10600 / 20000Checked up to n=14600 / 20000Checked up to n=14800 / 20000
Number type counts among wrong guesses:
Perfect square: 34
Semiprime: 1008
Powerful: 34
Palindromic: 13
Odd: 1048

Example numbers for each type:
Perfect square: [9, 25, 49, 121, 169, 289, 361, 529, 625, 841]
Semiprime: [9, 25, 49, 57, 87, 93, 121, 145, 169, 177]
Powerful: [9, 25, 49, 121, 169, 289, 361, 529, 625, 841]
Palindromic: [9, 121, 303, 393, 515, 535, 717, 10201, 12021, 13531]
Odd: [9, 25, 49, 57, 87, 93, 121, 145, 169, 177]

Wrong guess mapping (first 10 wrong guesses):
Wrong guess 9: n values [7, 8]
Wrong guess 25: n values [23, 24]
Wrong guess 49: n values [47, 48]
Wrong guess 57: n values [53]
Wrong guess 87: n values [83]
Wrong guess 93: n values [89]
Wrong guess 121: n values [113, 114, 115, 116, 117, 118, 119, 120]
Wr

In [1]:
import math
import sympy

def find_potential_prime(n, max_range_type='sqrt'):
    if max_range_type == 'sqrt':
        upper_bound = math.ceil(math.sqrt(n))
    elif max_range_type == 'n-sqrt':
        upper_bound = max(2, n - math.ceil(math.sqrt(n)))
    else:
        upper_bound = n
    numbers = list(range(2, upper_bound))
    remainders = set()
    for i in numbers:
        remainders.add(i - (n % i))
    m = 1 if n % 2 == 0 else 2
    while m in remainders:
        m += 2
    primetest = n + m
    is_prime = sympy.isprime(primetest)
    return primetest, is_prime, m

max_n = 20000  # You can adjust this value as needed
wrong_to_correct_map = {}
progress_step = max_n // 100 if max_n >= 100 else 1
for n in range(1, max_n + 1):
    primetest, is_prime, m = find_potential_prime(n, max_range_type='sqrt')
    if not is_prime:
        # Rerun with wider range
        correcttest, correct_is_prime, correct_m = find_potential_prime(n, max_range_type='n-sqrt')
        if correct_is_prime:
            wrong_to_correct_map[(n, primetest)] = (correcttest, correct_m)
    if n % progress_step == 0 or n == max_n:
        print(f"Checked up to n={n} / {max_n}", end='\r', flush=True)
print()

print("\nMapping of wrong guesses to correct guesses (first 20):")
for (n, wrong), (correct, correct_m) in list(wrong_to_correct_map.items())[:20]:
    print(f"n={n}: wrong_guess={wrong}, correct_guess={correct}, correct_m={correct_m}")

Checked up to n=20000 / 20000

Mapping of wrong guesses to correct guesses (first 20):
n=7: wrong_guess=9, correct_guess=11, correct_m=4
n=8: wrong_guess=9, correct_guess=11, correct_m=3
n=23: wrong_guess=25, correct_guess=29, correct_m=6
n=24: wrong_guess=25, correct_guess=29, correct_m=5
n=47: wrong_guess=49, correct_guess=53, correct_m=6
n=48: wrong_guess=49, correct_guess=53, correct_m=5
n=53: wrong_guess=57, correct_guess=59, correct_m=6
n=83: wrong_guess=87, correct_guess=89, correct_m=6
n=89: wrong_guess=93, correct_guess=97, correct_m=8
n=113: wrong_guess=121, correct_guess=127, correct_m=14
n=114: wrong_guess=121, correct_guess=127, correct_m=13
n=115: wrong_guess=121, correct_guess=127, correct_m=12
n=116: wrong_guess=121, correct_guess=127, correct_m=11
n=117: wrong_guess=121, correct_guess=127, correct_m=10
n=118: wrong_guess=121, correct_guess=127, correct_m=9
n=119: wrong_guess=121, correct_guess=127, correct_m=8
n=120: wrong_guess=121, correct_guess=127, correct_m=7
n=13

In [ ]:
import math
import sympy

def find_potential_prime_with_m_and_i(n, max_range_type='sqrt'):
    if max_range_type == 'sqrt':
        upper_bound = math.ceil(math.sqrt(n))
    elif max_range_type == 'n-sqrt':
        upper_bound = max(2, n - math.ceil(math.sqrt(n)))
    else:
        upper_bound = n
    numbers = list(range(2, upper_bound))
    remainders = set()
    for i in numbers:
        remainders.add(i - (n % i))
    m = 1 if n % 2 == 0 else 2
    i_values = []
    while m in remainders:
        m += 2                
        primetest = n + m
        is_prime = sympy.isprime(primetest)
        if is_prime:
            first_prime = primetest
            i_values.append(m)
    # Find all i values that would produce this m
    

    return primetest, is_prime, m, i_values

max_n = 20000  # You can adjust this value as needed
wrong_to_correct_map = {}
progress_step = max_n // 100 if max_n >= 100 else 1
for n in range(1, max_n + 1):
    primetest, is_prime, wrong_m, wrong_i_values = find_potential_prime_with_m_and_i(n, max_range_type='sqrt')
    if not is_prime:
        # Rerun with wider range
        correcttest, correct_is_prime, correct_m, correct_i_values = find_potential_prime_with_m_and_i(n, max_range_type='n-sqrt')
        if correct_is_prime:
            wrong_to_correct_map[(n, primetest)] = {
                'wrong_m': wrong_m,
                'correct_guess': correcttest,
                'correct_m': correct_m,
                'correct_i_values': correct_i_values
            }
    if n % progress_step == 0 or n == max_n:
        print(f"Checked up to n={n} / {max_n}", end='\r', flush=True)
print()

print("\nMapping of wrong guesses to correct guesses (first 20):")
for (n, wrong), info in list(wrong_to_correct_map.items())[:20]:
    print(f"n={n}: wrong_guess={wrong}, wrong_m={info['wrong_m']}, correct_guess={info['correct_guess']}, correct_m={info['correct_m']}, correct_i_values={info['correct_i_values']}")

Checked up to n=20000 / 20000

Mapping of wrong guesses to correct guesses (first 20):
n=7: wrong_guess=9, wrong_m=2, correct_guess=11, correct_m=4, correct_i_values=[]
n=8: wrong_guess=9, wrong_m=1, correct_guess=11, correct_m=3, correct_i_values=[]
n=23: wrong_guess=25, wrong_m=2, correct_guess=29, correct_m=6, correct_i_values=[]
n=24: wrong_guess=25, wrong_m=1, correct_guess=29, correct_m=5, correct_i_values=[]
n=47: wrong_guess=49, wrong_m=2, correct_guess=53, correct_m=6, correct_i_values=[]
n=48: wrong_guess=49, wrong_m=1, correct_guess=53, correct_m=5, correct_i_values=[]
n=53: wrong_guess=57, wrong_m=4, correct_guess=59, correct_m=6, correct_i_values=[]
n=83: wrong_guess=87, wrong_m=4, correct_guess=89, correct_m=6, correct_i_values=[]
n=89: wrong_guess=93, wrong_m=4, correct_guess=97, correct_m=8, correct_i_values=[]
n=113: wrong_guess=121, wrong_m=8, correct_guess=127, correct_m=14, correct_i_values=[]
n=114: wrong_guess=121, wrong_m=7, correct_guess=127, correct_m=13, corre